In [30]:
import xml.etree.ElementTree as ET
from xml.etree.ElementTree import ParseError
import pandas as pd
from glob import glob
import os.path as op
import numpy as np
from collections import Counter
import operator
from string import punctuation
from nltk import SnowballStemmer
from nltk import pos_tag
import nltk
import os

In [124]:
def eliminate_phrase(path, phrase):
    tree = ET.parse(path)
    root = tree.getroot()

    for problem in root:
        for child in problem:
            if child.tag == "question":
                child.text = child.text.replace(phrase, "")

    # Saving the result
    tree.write(path)

In [183]:
def add_Label_Calculatoir(path):
    tree = ET.parse(path)  
    root = tree.getroot()
    
    for problem in root:
        k=problem.find("label")
        if k is not None:
            continue
            
        for child in problem:
            if child.tag == "question":
                if "closest to" in child.text.split(".")[-1]:
                    #Add the tag "label"
                    attrib = {}  
                    label = problem.makeelement('label', attrib)  
                    label.text = "Calculatoir"
                    problem.append(label)
                    continue
                    
            
            if child.tag=="choices":
                if len(list(child))==0 :
                    continue
                result=child.find("choice").text
                    
                    
                result = ''.join([i for i in result if not i.isdigit()])
                s=0
                for choice in child:
                    if ''.join([i for i in choice.text if not i.isdigit()]) == result:
                        s+=1
                        if s==3:
                            #Add the tag "label"
                            attrib = {}  
                            label = problem.makeelement('label', attrib)  
                            label.text = "Calculatoir"
                            problem.append(label)
                            continue
                    else:
                        continue
                
                    
            
    #Saving the result            
    tree.write(path)

In [184]:
def add_Label_Definition(path):
    tree = ET.parse(path)  
    root = tree.getroot()
    
    for problem in root:
        k=problem.find("label")
        if k is not None:
            continue
            
        for child in problem:
            if child.tag == "question":
                if "defined as" in child.text.split(".")[-1]:
                    #Add the tag "label"
                    attrib = {}  
                    label = problem.makeelement('label', attrib)  
                    label.text = "Definition"
                    problem.append(label)
                    
            
    #Saving the result            
    tree.write(path)

In [185]:
def read_all_problems(file_path):
    # read all problems XML file and return a dataframe 
    xml_data = open(file_path, encoding="utf8").read()
    problems = ET.XML(xml_data)
    records = []
    for problem in problems:
        record = {}
        record['filename'] = problem.attrib['filename']
        record['topic'] = problem.attrib['topic']
        record['year'] = problem.attrib['year']
        for child in problem:
            if child.tag == "choices":
                for choice in child:
                    id = choice.attrib['id']
                    record[choice.tag + '_' + id] = choice.text
            else:
                record[child.tag] = child.text
                if child.tag == 'question':
                    record['question_nb'] = child.attrib['id']
        records.append(record)
    return pd.DataFrame(records)
        


In [186]:
path="..\Data\qa_mock_exams\all_problems_Labeled.xml"

file_paths = sorted(glob(op.join('..','Data', 'qa_mock_exams', '*.xml')))
for path in file_paths:
    print (path)

# Eliminate "Select exactly 1 answer(s) from the following:" from all questions
eliminate_phrase (path, "Select exactly 1 answer(s) from the following:")

add_Label_Calculatoir(path)
add_Label_Definition(path)


..\Data\qa_mock_exams\all_problems.xml
..\Data\qa_mock_exams\all_problems_Labeled.xml


In [189]:
problems_df = read_all_problems(path)
#print(problems_df.count())
problems_df.to_csv('all_problems_Labeled.csv')
problems_df.loc[problems_df['label'] == "Calculatoir"]
#problems_df[ problems_df.choice_D.notnull()]
#problems_df

,answer,choice_A,choice_B,choice_C,choice_D,comments,filename,label,question,question_nb,topic,year
19,C,"$320,000.","$423,000.","$1,176,000.","$1,552,000.","""The Time Value of Money,"" Richard A. Defusco,...",2008_part_1.xml,Calculatoir,"Rachel Kelly, age 24, is planning for retireme...",20,Quantitative Methods,2008
20,B,"($24,537.)","$10,558.","$49,825.","$93,929.","""Discounted Cash Flow Applications,"" Richard A...",2008_part_1.xml,Calculatoir,An analyst gathered the following information ...,21,Quantitative Methods,2008
21,B,12.5%.,20.8%.,25.0%.,41.7%.,"""Discounted Cash Flow Applications,"" Richard A...",2008_part_1.xml,Calculatoir,An analyst gathered the following information ...,22,Quantitative Methods,2008
23,B,Answer A,Answer B,Answer C,Answer D,"""Statistical Concepts and Market Returns,"" Ric...",2008_part_1.xml,Calculatoir,An analyst gathered the following information ...,24,Quantitative Methods,2008
24,A,6.83% 77.5,6.83% 96.8,7.68% 77.5,7.68% 96.8,"""Statistical Concepts and Market Returns,"" Ric...",2008_part_1.xml,Calculatoir,An analyst gathered the following annual retur...,25,Quantitative Methods,2008
25,A,1.36 0.52,1.36 7.44,1.53 0.52,1.53 7.44,"""Statistical Concepts and Market Returns,"" Ric...",2008_part_1.xml,Calculatoir,An analyst gathered the following information ...,26,Quantitative Methods,2008
29,C,15.0 to 69.0.,18.8 to 65.2.,34.3 to 49.7.,39.0 to 45.0.,"""Common Probability Distributions,"" Richard A....",2008_part_1.xml,Calculatoir,An analyst determined that the sample mean and...,30,Quantitative Methods,2008
34,C,13.,26.,60.,120.,"""Efficiency and Equity,"" Michael Parkin 2008 ...",2008_part_1.xml,Calculatoir,Venkat Reddy is very fond of mango fruits. If ...,35,Economics,2008
36,A,"$90,000","$110,000","$130,000","$150,000","""Organizing Production,"" Michael Parkin 2008 ...",2008_part_1.xml,Calculatoir,Choice Malts Breweries has compiled the follow...,37,Economics,2008
39,C,0.67.,0.72.,1.40.,1.50.,"""Elasticity,"" Michael Parkin 2008 Modular Lev...",2008_part_1.xml,Calculatoir,"In response to the influx of tourists, the dem...",40,Economics,2008


In [131]:
phrase = "An analyst gathered the following annual return information about a portfolio since its inception on 1 January 2003: Year Portfolio return 2003 8.6% 2004 11.2% 2005 12.9% 2006 15.1% 2007 -9.4% The portfolio's mean absolute deviation and variance of annual returns, respectively, for the five-year period are closest to: Mean absolute deviation Variance"
print(phrase.split(".")[-1])
if "closest to" in phrase.split(".")[-1]:
    print("true")

4% The portfolio's mean absolute deviation and variance of annual returns, respectively, for the five-year period are closest to: Mean absolute deviation Variance
true
